In [ ]:
pip install xlrd
pip install openpyxl
pip install pandas
pip install konlpy
pip install customized_konlpy
pip install matplotlib
pip install emoji
pip install soynlp
pip install konlpy
pip install torch transformers
pip install sentence-transformers
pip install numpy

In [ ]:
pip install customized_konlpy

In [2]:
import pandas as pd
import os
import numpy as np
import re

from emoji import core
import re

from konlpy.tag import Okt
from ckonlpy.tag import Twitter
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
# import seaborn as sns
# import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

c:\Users\Playdata\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_excel('data/카운터6.xlsx')
tag_list = df['Tag'].dropna().tolist()

print(tag_list)


['락', '추억', '회상', '잔잔한', '카페', '겨울', '연말', '크리스마스', '캐롤', '분위기', '댄스', '드라이브', '트로피컬', '일렉', '하우스', '힐링', '기분전환', '팝', '운동', '트렌드', '고백', '이별', '사랑', '짝사랑', '슬픔', '포크', '메탈', '스트레스', '새해', '여행', '우울', '감각', '힙합', '국내', '밤', '그루브', '새벽', '재즈', '가을', '감성', '설렘', '봄', '비', '혼자', '조용한', '맥주', '고민', '편안한', '어쿠스틱', '에너지', '달달', '남녀', '신나는', '미국', '랩소디', '클래식', '영화', '스윙', '상쾌', '아침', '휴식', '매장', '여름', '피트니스', '산책', '필라테스', '헬스', '다이어트', '러닝', '요가', '스포츠', '발라드', '위로', '옛날', '연주', '클럽', '오후', '주말', '나른함', '중국', '달', '유럽', '정화', '공기', '사무실', '알앤비', '불금', '차', '연애', '썸', '독서', '펍', '세련', '라운지', '퇴근', '한강', '랩', '섹시', '비트', '뉴에이지', '무드', '스웩', '꿀', '버스', '트렌디', '라디오', '벚꽃', '따뜻함', '포근함', '유튜브', '팝송', '자연', '프로그레시브', '바다', '시원', '아련', '치명', '강렬한', '여자', '아이돌', '인디뮤직', '호강', '인디밴드', '자장가', '추위', '일상', '잠', '몽환', '폭발', '페스티벌', '흥', '외힙', '기분', '키즈', '동화', '기타', '영국', '생각', '야경', '바캉스', '썸머', '휴가', '해변', '펑크', '고음', '명품', '연인', '아픔', '조깅', '출근', '소울', '가요', '여운', '인디음악', '힘든', '빠른', '퓨전', '해외', '저녁'

In [12]:
from sentence_transformers import SentenceTransformer

# SBERT 모델 로드
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# Okt 형태소 분석기
tokenizer = Twitter()
tokenizer.add_dictionary(tag_list, 'Noun')
tokenizer.add_dictionary(["플레이리스트", "집중력", "릴렉스", "고즈넉"], 'Noun')
# 미리 정해진 키워드의 BERT 임베딩 계산
predefined_embeddings = {kw: model.encode(kw) for kw in tag_list}

def extract_keywords(sentence, stopwords):
    # 형태소 분석
    sentence = core.replace_emoji(sentence, replace="")
    sentence = re.sub("[^ㄱ-ㅎ ㅏ-ㅣ가-힣 1-9]", "", sentence)  # 정규 표현식 수행
    sentence = re.sub('^ +', "", sentence)  # 공백은 empty 값으로 변경
    tokenized_sentence = tokenizer.pos(sentence)

    # 명사, 동사, 형용사 추출
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word[0] in stopwords]  # 불용어 제거
    print(stopwords_removed_sentence)
    keywords = [word for word, pos in stopwords_removed_sentence if pos in ['Noun', 'Adjective']]

    return keywords

def map_to_predefined_keywords(extracted_keywords):
    mapped_keywords = []
    for keyword in extracted_keywords:
        keyword_embedding = model.encode(keyword)
        similarities = {kw: cosine_similarity([keyword_embedding], [emb]).item() for kw, emb in predefined_embeddings.items()}
        mapped_keyword = max(similarities, key=similarities.get)
        mapped_keywords.append(mapped_keyword)
    return mapped_keywords

In [17]:
stopwords = [
    "이", "가", "은", "는", "을", "를", "의", "에", "에서", "와", "과", "도", "로", "으로", "하고",
    "나", "다", "라", "의거", "에게", "뿐", "만", "조차", "까지", "에서부터", "으로부터", "까지만",
    "그리고", "그러나", "하지만", "그렇지만", "그래서", "그러므로", "따라서", "그렇기 때문에",
    "만약", "만일", "비록", "뿐만 아니라", "혹은", "또는", "만약에", "하여", "나", "너", "그",
    "그녀", "저", "이", "그", "저", "우리", "너희", "그들", "것", "데", "수", "사람", "경우",
    "때", "아주", "매우", "너무", "더", "가장", "거의", "약간", "조금", "상당히", "정말",
    "진짜", "참", "자주", "항상", "또", "다시", "언제나", "이미", "아직", "곧", "방금", "지금",
    "현재", "일단", "당장", "즉시", "곧바로", "이것", "저것", "그것", "여기", "저기", "거기", "플레이리스트",
    "노래", "곡", "듣기", "좋은", "어울리는", "할", "때", "듣는", "했을", "지고", "음악", "나올", "트는", "추천", "찬", "면서",
    "앞", "고픈", "운", "시간", "가득", "선율", "여주", "높", "뮤직", "충만", "날", "위", "히트","모음",
    "후", "업", "필요", "순간", "후의", "타임", "멜로디", "속", "기분", "여", "명곡", "물씬", "적", "온", "소리", "트랙", "취향", "저격", "송"
]

In [18]:
# 입력 문장
sentence = "드라이브를 위한 로드 트립 송"

# 키워드 추출
extracted_keywords = extract_keywords(sentence, stopwords)
print(f"Extracted Keywords: {extracted_keywords}")

# 유사 키워드 매핑
mapped_keywords = map_to_predefined_keywords(extracted_keywords)
print(f"Mapped Keywords: {mapped_keywords}")

[('드라이브', 'Noun'), ('한', 'Josa'), ('로드', 'Noun'), ('트립', 'Noun')]
Extracted Keywords: ['드라이브', '로드', '트립']
Mapped Keywords: ['드라이브', '트랩', '최고']
